In [1]:
from itertools import combinations
import json
import os

import matplotlib.pyplot as plt
import numpy as np

from data import COUNTRIES, get_all_data_and_labels
from utils import make_comparable

In [2]:
CORRS_FILE = 'correlations.json'

In [3]:
all_data, all_labels = get_all_data_and_labels()

In [4]:
# For reference if you want to look up anything
print(all_labels)
print(COUNTRIES)

['bond_rets_local_fx', 'equity_prices', 'curr_acct_nom_usd', 'curr_acct_pct_gdp', 'fx_trd_wts_nom', 'fx_trd_wts_real', 'fx_vs_usd', 'gdp_nom', 'gdp_real', 'short_rates', 'long_rates', 'core_cpi_sa', 'm1_usd', 'm2_usd', 'm3_usd']
['USA', 'AUS', 'JPN', 'CAN', 'CHE', 'GBR', 'ESP', 'FRA', 'ITA', 'DEU']


In [22]:
# Load or compute correlations
if os.path.exists(CORRS_FILE):
    corrs = json.load(open(CORRS_FILE, 'r'))
    
else:
    corrs = {}

    for i, j in combinations(range(len(all_data)), r=2):
        if all_labels[i] not in corrs:
            corrs[all_labels[i]] = {}
        if all_labels[j] not in corrs[all_labels[i]]:
            corrs[all_labels[i]][all_labels[j]] = {}
        if all_labels[j] not in corrs:
            corrs[all_labels[j]] = {}
        if all_labels[i] not in corrs[all_labels[j]]:
            corrs[all_labels[j]][all_labels[i]] = {}

        inner_corrs = {}
        for country in COUNTRIES:
            if country not in all_data[i] or country not in all_data[j]:
                continue

            series1, series2 = make_comparable(all_data[i][country], all_data[j][country])
            inner_corrs[country] = np.corrcoef(series1, series2)[0,1]
            
        country_corrs_arr = np.array(list(inner_corrs.values()))
        inner_corrs['min'] = country_corrs_arr.min()
        inner_corrs['max'] = country_corrs_arr.max()
        inner_corrs['mean'] = country_corrs_arr.mean()
        inner_corrs['median'] = np.median(country_corrs_arr)
        inner_corrs['std'] = country_corrs_arr.std()

        corrs[all_labels[i]][all_labels[j]] = inner_corrs
        corrs[all_labels[j]][all_labels[i]] = inner_corrs
        
    json.dump(corrs, open(CORRS_FILE, 'w'), indent=4)

In [23]:
def print_combos_info(combos):
    for i, j in combos:
        print(all_labels[i], 'and', all_labels[j])
        for k, v in corrs[all_labels[i]][all_labels[j]].items():
            print('\t', k, v)

In [24]:
combos = combinations(range(len(all_data)), r=2)
sorted_combos = sorted(combos, key=lambda x: (
#     corrs[all_labels[x[0]]][all_labels[x[1]]]['min'] - corrs[all_labels[x[0]]][all_labels[x[1]]]['max'],
    corrs[all_labels[x[0]]][all_labels[x[1]]]['std'],
    abs(corrs[all_labels[x[0]]][all_labels[x[1]]]['median']),
    abs(corrs[all_labels[x[0]]][all_labels[x[1]]]['mean'])
))

In [25]:
print_combos_info(sorted_combos)

curr_acct_nom_usd and m2_usd
	 USA -0.6055232278008814
	 JPN 0.15249886809914273
	 CAN -0.6495946187345321
	 CHE 0.7952866508597232
	 GBR -0.873030957799283
	 ESP 0.08017436993117251
	 FRA -0.7760288342397174
	 ITA 0.03789927742774679
	 DEU 0.9442571822937192
	 min -0.873030957799283
	 max 0.9442571822937192
	 mean -0.09934014332921215
	 median 0.03789927742774679
	 std 0.6341610248716635
bond_rets_local_fx and fx_trd_wts_real
	 USA 0.507881553332733
	 AUS 0.7104508489886511
	 JPN -0.5927921561027397
	 CHE 0.7751816871747741
	 GBR -0.638171342531933
	 min -0.638171342531933
	 max 0.7751816871747741
	 mean 0.1525101181722971
	 median 0.507881553332733
	 std 0.6333970715462456
curr_acct_nom_usd and m1_usd
	 USA -0.47503810333174235
	 AUS -0.5290501576669767
	 JPN 0.2801567585222996
	 CAN -0.6381362317899845
	 CHE 0.7910327999518852
	 GBR -0.8609403331177339
	 ESP 0.5297284983346385
	 FRA -0.7354249769441923
	 ITA 0.04237376734109926
	 DEU 0.9202028948826193
	 min -0.8609403331177339
	 ma

	 CAN 0.9408254332487016
	 CHE 0.7403206922637222
	 GBR 0.9093654340151305
	 ESP 0.7359832421794706
	 FRA 0.9332951843572401
	 ITA 0.9065572571937708
	 DEU 0.938803205950826
	 min -0.39495334224856105
	 max 0.9410151366876969
	 mean 0.7390235826275553
	 median 0.9093654340151305
	 std 0.40854435346351625
fx_vs_usd and long_rates
	 USA -0.2939050639593958
	 AUS -0.2158029109495446
	 JPN 0.8053357275295764
	 CAN 0.2138262507530386
	 CHE 0.7037718258466679
	 GBR 0.3973919911884291
	 ESP -0.3645636184207854
	 FRA 0.16570558328920795
	 ITA -0.28500218207991085
	 DEU 0.4136973051622573
	 min -0.3645636184207854
	 max 0.8053357275295764
	 mean 0.15404549083595404
	 median 0.18976591702112328
	 std 0.406926296015313
fx_trd_wts_real and long_rates
	 USA -0.24932533466935397
	 AUS -0.2955623357635717
	 JPN 0.09781367774212363
	 CHE -0.594324534027889
	 GBR 0.5915554726013239
	 min -0.594324534027889
	 max 0.5915554726013239
	 mean -0.08996861082347345
	 median -0.24932533466935397
	 std 0.405407

	 min 0.5155984635231456
	 max 0.9985784858724247
	 mean 0.9326403780554637
	 median 0.9888465111211395
	 std 0.1479228401818595
long_rates and core_cpi_sa
	 USA -0.9439447623561001
	 AUS -0.910769427519607
	 JPN -0.8667721166469677
	 CAN -0.956863739300747
	 CHE -0.5997627308927143
	 GBR -0.9511426111908056
	 ESP -0.5653825425207438
	 FRA -0.9467437697965286
	 ITA -0.7501877928828913
	 DEU -0.9730803075758891
	 min -0.9730803075758891
	 max -0.5653825425207438
	 mean -0.8464649800682995
	 median -0.9273570949378536
	 std 0.1457466816973607
curr_acct_nom_usd and curr_acct_pct_gdp
	 USA 0.8632348556798672
	 AUS 0.4891689267160461
	 JPN 0.9676309837509186
	 CAN 0.8236038380891499
	 CHE 0.8339179980903384
	 GBR 0.9015345489571582
	 ESP 0.9649246681478681
	 FRA 0.9817855190596454
	 ITA 0.9842986025437589
	 DEU 0.9893127982386442
	 min 0.4891689267160461
	 max 0.9893127982386442
	 mean 0.8799412739273395
	 median 0.9332296085525131
	 std 0.14383217477764682
long_rates and m2_usd
	 USA -0.83

In [19]:
corrs = {}

for i, j in combinations(range(len(all_data)), r=2):
    print(all_labels[i], all_labels[j])
    if all_labels[i] not in corrs:
        corrs[all_labels[i]] = {}
    if all_labels[j] not in corrs[all_labels[i]]:
        corrs[all_labels[i]][all_labels[j]] = {}
    if all_labels[j] not in corrs:
        corrs[all_labels[j]] = {}
    if all_labels[i] not in corrs[all_labels[j]]:
        corrs[all_labels[j]][all_labels[i]] = {}

    inner_corrs = {}
    for country in COUNTRIES:
        if country not in all_data[i] or country not in all_data[j]:
            continue

        series1, series2 = make_comparable(all_data[i][country], all_data[j][country])
        inner_corrs[country] = np.corrcoef(series1, series2)[0,1]
        print('\t', country, inner_corrs[country])

    country_corrs_arr = np.array(list(inner_corrs.values()))
    inner_corrs['min'] = country_corrs_arr.min()
    inner_corrs['max'] = country_corrs_arr.max()
    inner_corrs['mean'] = country_corrs_arr.mean()
    inner_corrs['median'] = np.median(country_corrs_arr)

    corrs[all_labels[i]][all_labels[j]] = inner_corrs
    corrs[all_labels[j]][all_labels[i]] = inner_corrs

    print(f'min: {country_corrs_arr.min():.5}, '
          f'max: {country_corrs_arr.max():.5}, '
          f'mean: {country_corrs_arr.mean():.5}, '
          f'median: {np.median(country_corrs_arr):.5}, '
          f'std: {country_corrs_arr.std():.5}\n')
    
    
    

bond_rets_local_fx equity_prices
	 USA 0.9232704468431175
	 AUS 0.9028067918700352
	 JPN -0.25235974502761116
	 CAN 0.9518570331131264
	 CHE 0.9077346740217656
	 GBR 0.8622568094379277
	 ESP 0.5696051874361104
	 FRA 0.7874711891096722
	 ITA -0.04097923321371857
	 DEU 0.870197769118323
min: -0.25236, max: 0.95186, mean: 0.64819, median: 0.86623, std: 0.41312

bond_rets_local_fx curr_acct_nom_usd
	 USA -0.6729070841754053
	 AUS -0.39640826937303025
	 JPN 0.2536136832933748
	 CAN -0.5242046320903206
	 CHE 0.8404880829715451
	 GBR -0.849527672359138
	 ESP 0.3902074609291026
	 FRA -0.6424048329874917
	 ITA 0.36987818238183084
	 DEU 0.9244747997428251
min: -0.84953, max: 0.92447, mean: -0.030679, median: -0.071397, std: 0.62643

bond_rets_local_fx curr_acct_pct_gdp
	 USA -0.3084650546314134
	 AUS 0.4020044659185118
	 JPN 0.16419811251834387
	 CAN -0.07242835350385106
	 CHE 0.5354718880083617
	 GBR -0.6953224701671842
	 ESP 0.5745340637411191
	 FRA -0.6436558359571768
	 ITA 0.2873608101523073

	 USA -0.7279296118714181
	 AUS -0.6741948399920237
	 JPN 0.028012738863653068
	 CAN -0.5307531955747884
	 CHE 0.810357045974633
	 GBR -0.8464104009217649
	 ESP -0.20632099309045457
	 FRA -0.8048868582116739
	 ITA -0.3070352801674264
	 DEU 0.9465272853006935
min: -0.84641, max: 0.94653, mean: -0.23126, median: -0.41889, std: 0.61529

curr_acct_nom_usd gdp_real
	 USA -0.7823124795572656
	 AUS -0.6083576583603878
	 JPN 0.3732458966664205
	 CAN -0.4040999629803104
	 CHE 0.8175282177951586
	 GBR -0.8456623224349563
	 ESP -0.08418807695384334
	 FRA -0.6959740562503324
	 ITA -0.4831396752978523
	 DEU 0.9266721393967039
min: -0.84566, max: 0.92667, mean: -0.17863, median: -0.44362, std: 0.62712

curr_acct_nom_usd short_rates
	 USA 0.6582779619891255
	 AUS 0.4910400224397051
	 JPN -0.15768030884413675
	 CAN 0.352354583720039
	 CHE -0.656471125000812
	 GBR 0.7463014696187021
	 ESP -0.17715829540442538
	 FRA 0.5259352463035517
	 ITA -0.11773317296367736
	 DEU -0.8197622263756986
min: -0.81976, m

	 ESP -0.2425999997706852
	 FRA -0.39990592792170027
	 ITA -0.23554084530949193
	 DEU -0.06257905504676839
min: -0.86946, max: 0.25825, mean: -0.28335, median: -0.23907, std: 0.33147

fx_vs_usd short_rates
	 USA -0.40494884119780766
	 AUS -0.21452955687873546
	 JPN 0.829407698366391
	 CAN 0.19719472259188126
	 CHE 0.7314640210767434
	 GBR 0.46968122126756834
	 ESP -0.4900529352414739
	 FRA 0.11215719157569881
	 ITA -0.3747791808598308
	 DEU 0.42595613544469607
min: -0.49005, max: 0.82941, mean: 0.12816, median: 0.15468, std: 0.45857

fx_vs_usd long_rates
	 USA -0.2939050639593958
	 AUS -0.2158029109495446
	 JPN 0.8053357275295764
	 CAN 0.2138262507530386
	 CHE 0.7037718258466679
	 GBR 0.3973919911884291
	 ESP -0.3645636184207854
	 FRA 0.16570558328920795
	 ITA -0.28500218207991085
	 DEU 0.4136973051622573
min: -0.36456, max: 0.80534, mean: 0.15405, median: 0.18977, std: 0.40693

fx_vs_usd core_cpi_sa
	 USA 0.2499750206885645
	 AUS -0.2771765225643785
	 JPN 0.7739084493228843
	 CAN -0.2

	 FRA -0.842675660349267
	 ITA -0.6619892931563535
	 DEU -0.9078797559222055
min: -0.91599, max: -0.40647, mean: -0.75208, median: -0.84268, std: 0.18046

core_cpi_sa m1_usd
	 USA 0.881131514549202
	 AUS 0.9526096592484713
	 JPN 0.3692611297579412
	 CAN 0.9323759807277697
	 CHE 0.7024976580423159
	 GBR 0.9438587147002946
	 ESP 0.7768400007257077
	 FRA 0.9253815207149945
	 ITA 0.9582182223809371
	 DEU 0.9342439165452462
min: 0.36926, max: 0.95822, mean: 0.83764, median: 0.92888, std: 0.17558

core_cpi_sa m2_usd
	 USA 0.9326573211274893
	 JPN -0.22424759386552848
	 CAN 0.9346143326132155
	 CHE 0.7064306769104173
	 GBR 0.9289432517636669
	 ESP 0.8622398530482611
	 FRA 0.9411685847886273
	 ITA 0.9535396195031653
	 DEU 0.9635390454233593
min: -0.22425, max: 0.96354, mean: 0.77765, median: 0.93266, std: 0.36206

core_cpi_sa m3_usd
	 AUS 0.9410151366876969
	 JPN -0.39495334224856105
	 CAN 0.9408254332487016
	 CHE 0.7403206922637222
	 GBR 0.9093654340151305
	 ESP 0.7359832421794706
	 FRA 0.933